# Exercise 1 - Question 1 (Language Models)

In [2]:
#Requirements: to be installed in the virtual environment

# pip install torch
# pip install transformers
# pip install datasets
# pip install numpy

SyntaxError: invalid syntax (3281737530.py, line 3)

<a name="ngram_lm"></a>
## Task A: Data Exploration


Let's use an unsupervised dataset (raw corpus) to evaluate language models' perplexity. We use Huggingface's `datasets` library to download needed datasets.
 

Here we use the `Penn Treebank` dataset, featuring a million words of 1989 Wall Street Journal material. The rare words in this version are already replaced with `<unk>` token. The numbers are also replaced with a special token. This token replacement helps us to end up with a more reasonable vocabulary size to work with.


In [3]:
import numpy as np
import datasets
from datasets import load_dataset
import torch

ptb_dataset = load_dataset("ptb_text_only", split="train")

# splitting dataset in train/test (to be later used for language model evaluation)
ptb_dataset = ptb_dataset.train_test_split(test_size=0.2, seed=1)
ptb_train, ptb_test = ptb_dataset['train'], ptb_dataset['test']

/home/ezradin/.conda/envs/pip_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset ptb_text_only (/home/ezradin/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f)
Loading cached split indices for dataset at /home/ezradin/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f/cache-6d72a72d5c72be3d.arrow and /home/ezradin/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f/cache-3e2d0e2b1466183a.arrow


#### Let's have a look at a few samples of the training dataset (and also the structure of the dataset)

In [3]:
print(f"{ptb_train[0]}\n\n{ptb_train[1]}\n\n{ptb_train[2]}")

{'sentence': "a former executive agreed that the departures do n't reflect major problems adding if you see any company that grows as fast as reebok did it is going to have people coming and going"}

{'sentence': 'with talk today of a second economic <unk> in west germany east germany no longer can content itself with being the economic star in a loser league'}

{'sentence': 'transportation secretary sam skinner who earlier fueled the anti-takeover fires with his <unk> attacks on foreign investment in u.s. carriers now says the bill would further <unk> the jittery capital markets'}


During generation with a given language model, we often need to have a `<stop>` token in our vocabulary to terminate the generation of a given sentence/paragraph. In this dataset, every sample is a sentence, and the `<stop>` token should be added to the end of every sample (i.e., end of sentence).

#### Create a new train/test dataset starting from `ptb_train` and `ptb_test` that has a `<stop>` at the end of each sentence. (Note: do not change the structure of the datasets objects, and just change the respective sentences as discussed).
Hint: use the `.map()` functionality of the `datasets` package (read more [here](https://huggingface.co/docs/datasets/process#map])).

In [4]:
def add_stop_token(input_sample: dict):
    '''
    args:
        input_sample: a dict representing a sample of the dataset. (look above for the dict struture)
    output:
        modified_sample: modified dict adding <stop> at the end of each sentence.
    '''
    modified_sample ={}
    modified_sample['sentence'] = input_sample['sentence'] + " <stop>"
    
    
    return modified_sample
    
    
ptb_train = ptb_train.map(add_stop_token)
ptb_test = ptb_test.map(add_stop_token)

Loading cached processed dataset at /home/ezradin/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f/cache-9390bd991ee6f14d.arrow
Loading cached processed dataset at /home/ezradin/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f/cache-952de2249dda7f49.arrow


For the both `ptb_train` and `ptb_test` datasets, filter out every sample that has less than 3 tokens. it will help remove very short sentences that are not very helpful for training/evaluating a langugage model.

Hint: use `.filter()` functionality of the `datasets` package (read more [here](https://huggingface.co/docs/datasets/process#select-and-filter)).

In [5]:
ptb_train= ptb_train.filter(lambda dict: len(dict['sentence'])>3)
ptb_test= ptb_test.filter(lambda dict: len(dict['sentence'])>3)


Loading cached processed dataset at /home/ezradin/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f/cache-4dbad66706bdcdb0.arrow
Loading cached processed dataset at /home/ezradin/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f/cache-e30e090ecece4408.arrow


#### What are the 10 most frequent tokens in this dataset? Can you spot the token used to replace the numbers in this dataset? How are rare tokens replaced in this dataset?

In [6]:
from collections import Counter

count_tokens = Counter()

for sample in ptb_train:
    count_tokens.update(sample['sentence'].split())

count_tokens


Counter({'the': 40616,
         '<unk>': 35918,
         '<stop>': 33654,
         'N': 25966,
         'of': 19459,
         'to': 18896,
         'a': 16901,
         'in': 14473,
         'and': 14013,
         "'s": 7850,
         'for': 7108,
         'that': 7100,
         '$': 5957,
         'is': 5924,
         'it': 4870,
         'said': 4840,
         'on': 4494,
         'at': 3946,
         'by': 3939,
         'as': 3860,
         'from': 3818,
         'with': 3678,
         'million': 3655,
         'mr.': 3468,
         'was': 3245,
         'be': 3122,
         'are': 3111,
         'its': 3089,
         'he': 2898,
         'but': 2841,
         'has': 2818,
         'an': 2787,
         "n't": 2694,
         'will': 2582,
         'have': 2557,
         'new': 2236,
         'company': 2175,
         'or': 2141,
         'they': 2075,
         'this': 1953,
         'year': 1908,
         'which': 1872,
         'would': 1840,
         'about': 1732,
         'says'

The token used to replace the numbers in this dataset: 'N'
The token used to replace the rare tokens in this dataset: 'unk'




## Task B: Fixed-Window Neural Language Models <a name='fixed_window_neural_lm'></a>

This language model take as input a constant number of tokens, and then outputs a probability distribution for the next token. In this section, we assume the underlying model is a FeedForward Network (FFN) with a single hidden layer. This model doesn't have the sparsity issue of N-gram language models, but is always limited to a fixed window of tokens.

In this section, we don't include the training of the model but rather we use a pretrained model on the same training dataset. We evaluate the language model over the `ptb_test` dataset, to show the power of neural language models, when compared to N-gram language models.

More importantly, we use PyTorch modules in this section, so that you get more familiar with its capabilities. Throughout this exercise, we use a `window_size=3` for this model.



Let's first create a dataset of all consecutive tokens of length `window_size` from the `ptb_train` dataset. you can read more about PyTorch datasets and how to create a custom dataset  [here](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files).

In [7]:
from torch.utils.data import Dataset, DataLoader

window_size = 3
vocabulary_size = 10000
word_emb_dim = 100
hidden_dim = 100


class FixedWindowDataset(Dataset):
    # read more about custom datasets at https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
    def __init__(self,
                 train_dataset: datasets.arrow_dataset.Dataset,
                 test_dataset: datasets.arrow_dataset.Dataset,
                 window_size: int,
                 vocabulary_size: int
                ):
        self.prepared_train_dataset = self.prepare_fixed_window_lm_dataset(train_dataset, window_size + 1)
        self.prepared_test_dataset = self.prepare_fixed_window_lm_dataset(test_dataset, window_size + 1)
        
        dataset_vocab = self.get_dataset_vocabulary(train_dataset)
        # defining a dictionary that simply maps tokens to their respective index in the embedding matrix
        self.word_to_index = {word: idx for idx,word in enumerate(dataset_vocab)}
        self.index_to_word = {idx: word for idx,word in enumerate(dataset_vocab)}
        
        assert vocabulary_size > len(dataset_vocab) , f"The dataset vocab size is {len(dataset_vocab)}!"

    def __len__(self):
        return len(self.prepared_train_dataset)
    
    def get_encoded_test_samples(self):
        all_token_lists = [sample.split() for sample in self.prepared_test_dataset]
        all_token_ids = [[self.word_to_index.get(word, self.word_to_index["<unk>"])
                          for word in token_list[:-1]]
                         for token_list in all_token_lists
                        ]
        all_next_token_ids = [self.word_to_index.get(token_list[-1], self.word_to_index["<unk>"]) for 
                              token_list in all_token_lists]
        return torch.tensor(all_token_ids), torch.tensor(all_next_token_ids)
        
    def __getitem__(self, idx):
        # here we need to transform the data to the format we expect at the model input
        token_list = self.prepared_train_dataset[idx].split()
        # having a fallback to <unk> token if an unseen word is encoded.
        token_ids = [self.word_to_index.get(word, self.word_to_index["<unk>"]) for word in token_list[:-1]]
        next_token_id = self.word_to_index.get(token_list[-1], self.word_to_index["<unk>"])
        return torch.tensor(token_ids), torch.tensor(next_token_id)
    
    def decode_idx_to_word(self, token_id):
        return [self.index_to_word[id_.item()] for id_ in token_id]
    
    def get_dataset_vocabulary(self, train_dataset: datasets.arrow_dataset.Dataset):
        vocab = sorted(set(" ".join([sample["sentence"] for sample in train_dataset]).split()))
        # we also add a <start> token to include initial tokens in the sentences in the dataset
        vocab += ["<start>"]
        return vocab
    
    @staticmethod
    def prepare_fixed_window_lm_dataset(target_dataset: datasets.arrow_dataset.Dataset,
                                        window_size: int):
        '''
        Please note that for the very first tokens, they will be added like "<start> <start> Token#1".
        args:
            target_dataset: the target dataset where its consecutive tokens of length 'window_size' should be extracted
            window_size: the window size for the language model
        output:
            prepared_dataset: a list of strings each containing 'window_size' tokens.
        '''
        
        prepared_dataset = []
        for sample in target_dataset:
            sentence = "<start> "*(window_size-1) + sample['sentence']
            sentence=sentence.split()
            for idx in range(0,len(sentence)-window_size+1):
                merge_tokens = " ".join([sentence[j] for j in range(idx,idx+window_size)])
                prepared_dataset.append(merge_tokens)        
        return prepared_dataset
        
        

In [8]:
fixed_window_dataset = FixedWindowDataset(ptb_train, ptb_test, window_size, vocabulary_size)

# let's create a simple dataloader for this dataset
train_dataloader =  DataLoader(fixed_window_dataset, batch_size=8, shuffle=True)

In [9]:
fixed_window_dataset.prepared_test_dataset

['<start> <start> <start> jefferies',
 '<start> <start> jefferies group',
 '<start> jefferies group inc.',
 'jefferies group inc. said',
 'group inc. said third-quarter',
 'inc. said third-quarter net',
 'said third-quarter net income',
 'third-quarter net income fell',
 'net income fell N',
 'income fell N N',
 'fell N N to',
 'N N to $',
 'N to $ N',
 'to $ N million',
 '$ N million or',
 'N million or N',
 'million or N cents',
 'or N cents a',
 'N cents a share',
 'cents a share from',
 'a share from $',
 'share from $ N',
 'from $ N million',
 '$ N million or',
 'N million or N',
 'million or N cents',
 'or N cents a',
 'N cents a share',
 'cents a share on',
 'a share on more',
 'share on more shares',
 'on more shares a',
 'more shares a year',
 'shares a year earlier',
 'a year earlier <stop>',
 '<start> <start> <start> keeping',
 '<start> <start> keeping the',
 '<start> keeping the mood',
 'keeping the mood light',
 'the mood light the',
 'mood light the two',
 'light the two 

Now, let's define the underlying PyTorch model for the language model. You can read more about PyTorch models [here](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html).

**Note**: Here in the forward pass, we compute the negative log-likelihood after passing through the FFN layers. Here we use `torch.nn.LogSoftmax`, as it's numerically more stable than doing seperately `softmax` followed by taking its logarithm.

In [10]:
import torch.optim as optim

class Fixed_window_language_model(torch.nn.Module):
    def __init__(self, emb_dim, hidden_dim, window_size, vocab_size=10000):
        super().__init__()

        self.window_size = window_size
        self.emb_dim = emb_dim
        self.word_embeddings = torch.nn.Embedding(vocab_size, emb_dim) # word embeddings
        self.linear1 = torch.nn.Linear(window_size * emb_dim, hidden_dim) # first linear layer
        self.activation_func = torch.tanh # the activation function
        self.linear2 = torch.nn.Linear(hidden_dim, vocab_size) # second linear layer
        
        self.log_softmax = torch.nn.LogSoftmax(dim=1)
        self.criterion = torch.nn.NLLLoss()
     
    def forward(self, input_ids, labels):
        inputs_embeds = self.word_embeddings(input_ids)
        concat_input_embed = inputs_embeds.reshape(-1, self.emb_dim * self.window_size)
        hidden_state = self.activation_func( self.linear1(concat_input_embed) )
        logits = self.log_softmax( self.linear2(hidden_state) )
        loss = self.criterion(logits, labels)
        
        return loss

Now let's see how easy it is to train a model with PyTorch! (we provide a trained model in the cell after train, so that you can just start using the model without going through the time-consuming training)

In [11]:
# defining the model
model_fixed_window = Fixed_window_language_model(emb_dim=word_emb_dim, hidden_dim=hidden_dim,
                                                 window_size=window_size, vocab_size=vocabulary_size)

# defining the optimizer
optimizer = optim.SGD(model_fixed_window.parameters(),
                      lr=0.005,
                      momentum=0.9)

In [59]:
import os


for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        # get the inputs; data is a tuple of (context, target)
        context, target = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        loss = model_fixed_window(context, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5000 == 4999. :    # print every 5000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 5000:.3f}')
            running_loss = 0.0

print('Finished Training')

path_to_save = "/home/ezradin/NLP/nlu_ex1/models"
# saving the trained model
torch.save(model_fixed_window.state_dict(), os.path.join(path_to_save,"fixed_window_model.pt"))

[1,  5000] loss: 6.795
[1, 10000] loss: 6.280
[1, 15000] loss: 6.132
[1, 20000] loss: 6.053
[1, 25000] loss: 5.975
[1, 30000] loss: 5.944
[1, 35000] loss: 5.929
[1, 40000] loss: 5.887
[1, 45000] loss: 5.849
[1, 50000] loss: 5.813
[1, 55000] loss: 5.812
[1, 60000] loss: 5.783
[1, 65000] loss: 5.762
[1, 70000] loss: 5.757
[1, 75000] loss: 5.753
[1, 80000] loss: 5.756
[1, 85000] loss: 5.675
[1, 90000] loss: 5.716
[2,  5000] loss: 5.537
[2, 10000] loss: 5.594
[2, 15000] loss: 5.577
[2, 20000] loss: 5.566
[2, 25000] loss: 5.563
[2, 30000] loss: 5.533
[2, 35000] loss: 5.549
[2, 40000] loss: 5.535
[2, 45000] loss: 5.531
[2, 50000] loss: 5.520
[2, 55000] loss: 5.517
[2, 60000] loss: 5.521
[2, 65000] loss: 5.510
[2, 70000] loss: 5.527
[2, 75000] loss: 5.535
[2, 80000] loss: 5.541
[2, 85000] loss: 5.501
[2, 90000] loss: 5.477
Finished Training


We provide a trained model, so that you can start using it right away

In [13]:
fixed_window_checkpoint_file = "fixed_window_model.pt"
model_fixed_window.load_state_dict(torch.load(fixed_window_checkpoint_file)) 

<All keys matched successfully>

In [14]:
# context and 'target' ids (target is the next word after the context)
test_token_ids, test_target_ids = fixed_window_dataset.get_encoded_test_samples()  

We now have the `test_token_ids`, `test_target_ids` tensors for the test dataset. The `test_token_ids` are the context ids and `test_target_ids` are the respective **next token** (a.k.a. target here) for these contexts.
#### Using the trained model, implement a function that can output the loss for the discussed test dataset. How can we generally decide if the model is overfitted to the train dataset or not?

In [48]:
def generate_test_dataset_loss(model: torch.nn.Module,
                               test_token_ids: torch.Tensor, 
                               test_target_ids: torch.Tensor):
    '''
    args:
        model: fixed-window language model
        test_token_ids: the context ids in a single tensor.
        test_target_ids: the target ids (next token after the context) in a single tensor.
    output:
        avg_test_loss: The average loss of model over test dataset.
    '''
    batch_size = 4
    test_loss = []
    # Splitting the tensor into batches along the 0th dimension
    test_token_ids_batch = torch.split(test_token_ids, batch_size, dim=0)    
    test_target_ids_batch = torch.split(test_target_ids, batch_size, dim=0)    

    for tokens_batch,target_batch in zip(test_token_ids_batch,test_target_ids_batch):
        output_model = model(tokens_batch,target_batch)
        test_loss.append(output_model)

    avg_test_loss =  sum(test_loss)/len(test_loss)
    return avg_test_loss


test_dataset_loss = generate_test_dataset_loss(model_fixed_window, test_token_ids, test_target_ids)
print(f"Test dataset loss is {test_dataset_loss}")

Test dataset loss is 5.5500168800354


#### Using the trained fixed-window model, implemention a function that can output entropy for a given sequence.

In [66]:
import math
import torch

def get_seqeuence_entropy_fixed_window_lm(model: torch.nn.Module,
                                            input_sequence: str,
                                            window_size: int,
                                            word_to_idx: dict):
    '''
    Note that e.g., in order to get the first token probability, you need to pass a sequence
    like "<start> <start> <start>" (prefix padding) to the neural model. In a similar fashion, we need to pass
    "<start> <start> TOKEN#1" for getting the probability of the second token.
    args:
        model: fixed-window language model
        input_sequence: the sequence for which we want to calculate the probability
        window_size: the size of window for the language model
        word_to_idx: a mapping from words to the embedding indices (to encode tokens before being
                     passed to model). You can get this dict from 'fixed_window_dataset.word_to_index'
    output:
        sequence_entropy: the entropy for the input sequence using the trained model
    '''
    sequence_entropy=0
    input_sequence = "<start> "*(window_size) + input_sequence
    input_sequence = input_sequence.split()
    for idx in range(0,len(input_sequence)-window_size):
        #convert word to index
        context_tokens = [word_to_idx.get(input_sequence[j], word_to_idx['<unk>']) for j in range(idx,idx+window_size)]
        target_tokens = word_to_idx.get(input_sequence[idx+window_size],word_to_idx['<unk>'])
        # Convert target_tokens and context_token to tensors
        target_tokens_tensor = torch.tensor(target_tokens).unsqueeze(0)
        context_token_tensor = torch.tensor(context_tokens).unsqueeze(0)
        
        
        loss = model(context_token_tensor,target_tokens_tensor)
        context_token_prob = math.exp(-loss.item())

        sequence_entropy += -math.log2(context_token_prob)*context_token_prob

    return sequence_entropy

In [69]:
word_to_idx = fixed_window_dataset.word_to_index

def compute_perplexity(model, dataset, window_size, word_to_idx):
    total_entropy = 0.0
    total_tokens = 0

    for sample in dataset:
        input_sequence = sample['sentence']
        sequence_entropy = get_seqeuence_entropy_fixed_window_lm(model, input_sequence, window_size, word_to_idx)
        sequence_length = len(input_sequence.split()) - window_size + 1

        total_entropy += sequence_entropy
        total_tokens += sequence_length

    average_entropy = total_entropy / total_tokens
    perplexity = 2 ** average_entropy

    return perplexity


# Assuming you have the trained model, ptb_test dataset, window_size, and word_to_idx dictionary

perplexity = compute_perplexity(model_fixed_window, ptb_test, window_size, word_to_idx)
print(f"The fixed-window model perplexity over test dataset is {perplexity:.4f}")

The fixed-window model perplexity over test dataset is 1.0995


### Task C: RNN-based Language Model <a name='rnn_lm'></a>
To address the need for a neural architecture that can proceed with any length input (as opposed to the fixed-window model that can only process a fixed number of tokens), we implement the Recurrent Neural Network (RNN). The core idea behind is that we can apply the same weight W repeatedly.

An advatange of RNN model compared to fixed-window langauage model is that we can pass a given sentence at once, instead of passing it in many windows of size `window_size`. Moreover, the language model has the ability to look behind further that a fixed number of tokens.

 As we already did a neural model training exercise for the previous neural model, we only provide a trained LM at this section, so that you can focus only on the analysis part.
 
You can find the dataset structure as well as the RNN architecture in the `rnn_utils.py` file.

In [5]:
from rnn_utils import RNNDataset, RNN_language_model

ptb_train, ptb_test = ptb_dataset['train'], ptb_dataset['test']

vocabulary_size = 10000
word_emb_dim = 200
hidden_dim = 200

rnn_dataset = RNNDataset(ptb_train, ptb_test, vocabulary_size)

# if gpu is available, we puts the model on it 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Here we need a <pad> token for the RNN model, in order to have a batch of sequences with difference sizes 
pad_idx = rnn_dataset.pad_idx # the index for <pad> token
rnn_model = RNN_language_model(vocab_size=vocabulary_size, emb_dim=word_emb_dim, hidden_dim=hidden_dim,
                               pad_idx=pad_idx)
rnn_model.to(device)

RNN_language_model(
  (criterion): CrossEntropyLoss()
  (embedding): Embedding(10000, 200)
  (rnn): RNN(200, 200, num_layers=4)
  (dropout): Dropout(p=0.001, inplace=False)
  (lm_decoder): Linear(in_features=200, out_features=10000, bias=True)
)

load the model weights using the state_dict in `rnn_model.pt` file.

In [6]:
rnn_checkpoint_file = "rnn_model.pt"
rnn_model.load_state_dict(torch.load(rnn_checkpoint_file))

<All keys matched successfully>

As the training of an RNN model is time-consuming, we provide a trained language model on this dataset (`rnn_model.pt`), so that you can just analyze the model performance here.
As mentioned above, as RNN can get sequences with varying lengths, the input sequences should be padded with a special token like `<pad>`, so that we can create a batch of sentences. The output of the defined RNN model (see the architecture detail `rnn_utils.py`) is the model's entropy over the input data.

#### First get the encoded test samples of `ptb_test` dataset, and then pass these (already padded) sentences to the RNN model to get the respective entropy values. Compute the perplexity of the model and compare it with previous approaches.
**HINT**: You can use the `get_encoded_test_samples` function of `rnn_dataset` to get encoded test samples.


In [25]:
from tqdm import tqdm
import math

test_perplexity = -1
encode_test = rnn_dataset.get_encoded_test_samples().to(device)

sequence_entropy_rnn = 0
total_entropy_rnn = 0.0
total_tokens_rnn = 0

for i in tqdm(range(len(encode_test)-1)):

    loss = rnn_model(encode_test[i:i+1])
    context_token_prob = math.exp(-loss.item())
    sequence_entropy_rnn += -math.log2(context_token_prob)*context_token_prob

    total_entropy_rnn += sequence_entropy_rnn
    seq_length = (encode_test[0:1] != pad_idx).sum().item()
    total_tokens_rnn += seq_length

average_entropy_rnn = total_entropy_rnn / total_tokens_rnn
test_perplexity = 2 ** average_entropy_rnn

# Disable gradient computation
with torch.no_grad():
    # Forward pass
    output = rnn_model(encode_test[0:2])
print(output)
print(f"The model perplexity is {test_perplexity}")

100%|██████████| 8413/8413 [00:47<00:00, 175.34it/s]

tensor(6.7940, device='cuda:0')
The model perplexity is 2.4427237009778393


### Task D: MLM Transformer Language Models (Bonus Question: 10 pts) <a name='rnn_lm'></a>

We are here interested in computing the perplexity of MLM Transformer Language Models such as BERT and RoBERTa. Hoewever, the perplexity for MLM models is not well-defined (The difference with GPT models is illstrated [here](https://huggingface.co/docs/transformers/perplexity).

Instructions: First clone the following repository: https://github.com/asahi417/lmppl.
Install the requirements and follow the instructions to compute the pseudo-perplexity [(Wang and Cho, 2019)](https://aclanthology.org/W19-2304.pdf) of 'BERT-base-uncased', 'BERT-large-uncased', 'RoBERTa-base' and 'RoBERTa-large' for the sentences:
'Shelly ate the sliced banana with a fork' and 'The fork ate the sliced banana'.

Which sentence gets the lowest pseudo-perplexity for each of the models? Which is the best model according to this test?
What is the relation of this test to semantic roles?



In [ ]:
# YOUR CODE HERE

# YOUR ANSWERS HERE